<a href="https://colab.research.google.com/github/kperv/summarizer_app/blob/main/Summarization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ***Extractive summarization task***

### Main highlights

*   Supported languages are Russian and Spanish.
*   Text is broken into sentences by Spacy.
*   Sentences are tokenized by pretrained BertTokenizer.
*   Framework PyTorch (PyTorch Lightning)


### Neural Net architecture

The attempted idea is to train a centroid in a vector space by using Inception-like Convolutional Layers.

### Benchmark

Apply clustering algorithm on aggregated word vectors to get the result fast.

### Metric is Bert_Score


*   Supports target languages
*   Based on vector distance, which is better suited for the task



# **Installations and imports**

In [2]:
%%capture
!pip install transformers[sentencepiece]
!pip install pytorch-lightning
!pip install -U bert_score
!pip install datasets
!pip install https://huggingface.co/spacy/ru_core_news_md/resolve/main/ru_core_news_md-any-py3-none-any.whl
!pip install https://huggingface.co/spacy/es_core_news_md/resolve/main/es_core_news_md-any-py3-none-any.whl


import pytorch_lightning as pl
from pytorch_lightning.callbacks import LearningRateMonitor, ModelCheckpoint
from datasets import load_dataset
from transformers import BertTokenizer, BertModel
import spacy
import bert_score
from bert_score import score
import numpy as np
import os
import torch
from torch import nn
import torch.nn.functional as F
from torchvision import transforms
from torch.utils.data import DataLoader

In [ ]:
pl.seed_everything(42)
tokenizer = BertTokenizer.from_pretrained("bert-base-cased")
nlp_ru = spacy.load("ru_core_news_md")
nlp_es = spacy.load("es_core_news_md")

## requirements

In [17]:
print("Spacy = ", spacy.__version__)
print("torch = ", torch.__version__)
print("numpy = ", np.__version__)
print("datasets = ", datasets.__version__)
print("Transformers = ", transformers.__version__)
print("PyTorch Lightning = ", pl.__version__)
print("Spacy = ", spacy.__version__)
print("Bert Score = ", bert_score.__version__)

Spacy =  3.1.1


## Examples and checks of Bert score

Predictions correspond to a news headline and references to a ferst paragraph of the same article. Expected to get high scores.

In [8]:
predictions = ["Аналитик прокомментировал предстоящий визит Меркель в Москву"]
references = ["Ведущий научный сотрудник Центра германских исследований Института Европы РАН Александр Камкин прокомментировал в беседе с RT сообщение о том, что канцлер Германии Ангела Меркель и президент России Владимир Путин проведут переговоры в Москве 20 августа"]
P, R, F1 = score(predictions, references, lang='ru')
print(f"System level F1 score: {F1.mean():.3f}")
print(f"System level P score: {P.mean():.3f}")
print(f"System level R score: {R.mean():.3f}")

System level F1 score: 0.680
System level P score: 0.738
System level R score: 0.632


In [9]:
predictions = ["Qué sabemos sobre el diálogo entre la oposición y Maduro que está programado para comenzar este viernes en México"]
references = ["Tras años de tensiones, protestas y negociaciones estancadas, en medio de una situación económica muy deteriorada y complicada aún más por la pandemia de covid-19, el gobierno de Venezuela y la oposición intentará por quinta vez llegar a una solución para la crisis política mediante un diálogo, esta vez en México."]
P, R, F1 = score(predictions, references, lang='es')
print(f"System level F1 score: {F1.mean():.3f}")
print(f"System level P score: {P.mean():.3f}")
print(f"System level R score: {R.mean():.3f}")

System level F1 score: 0.636
System level P score: 0.663
System level R score: 0.611


## **Dataset is large-scale MultiLingual SUMmarization dataset**

### Main info:

**es**

Size of downloaded dataset files: 489.53 MB

Size of the generated dataset: 1274.55 MB

Total amount of disk used: 1764.09 MB

**Data Splits:** Train 266367	Val 10358	Test 1392



**ru**

Size of downloaded dataset files: 101.30 MB

Size of the generated dataset: 263.38 MB

Total amount of disk used: 364.68 MB

**Data Splits:** Train 25556	Val 750	Test 757

In [11]:
%%capture
dataset_ru = load_dataset("mlsum", "ru")
dataset_es = load_dataset("mlsum", "es")

Reusing dataset mlsum (/root/.cache/huggingface/datasets/mlsum/ru/1.0.0/77f23eb185781f439927ac2569ab1da1083195d8b2dab2b2f6bbe52feb600688)


Downloading:   0%|          | 0.00/27.4M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/19.5M [00:00<?, ?B/s]

0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

Dataset mlsum downloaded and prepared to /root/.cache/huggingface/datasets/mlsum/es/1.0.0/77f23eb185781f439927ac2569ab1da1083195d8b2dab2b2f6bbe52feb600688. Subsequent calls will reuse this data.


In [10]:
dataset_ru.num_rows

{'test': 757, 'train': 25556, 'validation': 750}

# ***Clustering solution***

In [ ]:
def gen_sentences(text):
  doc = nlp(text)
  assert doc.has_annotation("SENT_START")
  sentences = [str(sent) for sent in doc.sents]
  return sentences

def get_lens(sentences):
  return [len(sent) for sent in sentences]


In [ ]:
sample_ru_text = dataset_ru["train"]["text"][2]
sample_es_text = dataset_es["train"]["text"][2]

'Сладострастник в течение трех лет преследовал подростка в надежде совратить его. Как сообщили “МК” в следственном отделе по Хорошевскому району СУ СК при Прокуратуре РФ по Москве, 26 августа 2006 года 13-летний Павел вместе с другом отдыхал на берегу Москвы–реки рядом с Крылатским мостом. Там к ребятам подошел мужчина. Новый знакомый представился Евгением и предложил вместе пообедать в ресторане быстрого питания, а потом искупаться. Именно там, на берегу, педагог начал приставать к мальчику. Школьник убежал, но педофил успел снять голого подростка на мобильный телефон. После этого жизнь мальчика превратилась в сущий ад. Евгений узнал, где живет Павел, и стал шантажировать его. Этот кошмар продолжался три года. Преподаватель угрожал показать фотографию друзьям и знакомым Павла. Негодяй исписал непотребными надписями стены подъезда, где проживали друзья школьника. В один из дней он приехал в Сергиев Посад, к бабушке мальчика, и там накинулся на школьника с ножом. Наконец, отчаявшийся по

In [ ]:
lens = []
sentences = gen_sentences(sample_ru_text)
for sent in sentences:
  print(sent)
  lens.append(len(sent))
sum(lens)

Сладострастник в течение трех лет преследовал подростка в надежде совратить его.
Как сообщили “МК” в следственном отделе по Хорошевскому району СУ СК при Прокуратуре РФ по Москве, 26 августа 2006 года 13-летний Павел вместе с другом отдыхал на берегу Москвы–реки рядом с Крылатским мостом.
Там к ребятам подошел мужчина.
Новый знакомый представился Евгением и предложил вместе пообедать в ресторане быстрого питания, а потом искупаться.
Именно там, на берегу, педагог начал приставать к мальчику.
Школьник убежал, но педофил успел снять голого подростка на мобильный телефон.
После этого жизнь мальчика превратилась в сущий ад.
Евгений узнал, где живет Павел, и стал шантажировать его.
Этот кошмар продолжался три года.
Преподаватель угрожал показать фотографию друзьям и знакомым Павла.
Негодяй исписал непотребными надписями стены подъезда, где проживали друзья школьника.
В один из дней он приехал в Сергиев Посад, к бабушке мальчика, и там накинулся на школьника с ножом.
Наконец, отчаявшийся под

1798

In [ ]:
lens = []
sentences = gen_sentences(sample_es_text)
for sent in sentences:
  print(sent)
  lens.append(len(sent))
sum(lens)

In [ ]:
sentences[1]

'Как сообщили “МК” в следственном отделе по Хорошевскому району СУ СК при Прокуратуре РФ по Москве, 26 августа 2006 года 13-летний Павел вместе с другом отдыхал на берегу Москвы–реки рядом с Крылатским мостом.'

In [ ]:
import transformers
import torch

tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')
model = BertModel.from_pretrained("bert-base-multilingual-cased")
encoded_sentences = tokenizer(sentences)
encoded_sequences = tokenizer(text)

In [ ]:
tokenized_sents = []
ids = []
for sent in sentences:
  tokens = tokenizer.tokenize(sent)
  tokenized_sents.append(tokens)
  id = tokenizer.convert_tokens_to_ids(tokens)
  ids.append(id)


In [ ]:
input_ids = torch.tensor([ids[0]])
input_ids

tensor([[  526, 49238, 33580, 56680, 43372,   543, 21148, 83370, 12773, 38494,
         73759, 35973, 11429, 44181, 27100,   543, 12614, 49867, 12265, 98903,
         44678, 11258, 10933,   119]])

In [ ]:
encoded_sentences = tokenizer(sentences, padding=True, truncation=True, return_tensors="pt")
encoded_sentences.input_ids.shape

torch.Size([22, 63])

In [ ]:
decoded_string = tokenizer.decode(encoded_sentences.input_ids[1])

In [ ]:
decoded_string

'[CLS] Как сообщили [UNK] МК [UNK] в следственном отделе по Хорошевскому району СУ СК при Прокуратуре РФ по Москве, 26 августа 2006 года 13 - летний Павел вместе с другом отдыхал на берегу Москвы [UNK] реки рядом с Крылатским мостом. [SEP]'

In [ ]:
outputs = model(encoded_sentences.input_ids)
outputs.last_hidden_state.shape

torch.Size([22, 63, 768])

In [ ]:
embeddings = outputs.last_hidden_state

In [ ]:
def tokenize_function(example):
    return tokenizer(example["sentence1"], example["sentence2"], truncation=True)

## *Check for long training*

In [ ]:
!nvidia-smi

In [11]:
device = torch.device("cuda:0") if torch.cuda.is_available() else torch.device("cpu")
device

device(type='cpu')

# ***Neural Net solution***

In [ ]:
class MlSumDataModule(pl.LightningDataModule):
  def __init__(self, batch_size=32):
    super.__init__()
    self.batch_size = batch_size

  def prepare_data(self):
    self.dataset_ru = load_dataset("mlsum", "ru")
    self.dataset_es = load_dataset("mlsum", "es")
    nlp_ru = spacy.load("ru_core_news_md")
    nlp_es = spacy.load("es_core_news_md")

  def train_dataloader(self):
    dataset_ru_train = DataLoader(
        self.dataset_ru["train"], 
        batch_size=self.batch_size)
    dataset_es_train = DataLoader(
        self.dataset_es["train"], 
        batch_size=self.batch_size)
    loaders = [dataset_ru_train, dataset_es_train]
    return train_loaders

  def val_dataloader(self):
    dataset_ru_val = DataLoader(
        self.dataset_ru["val"], 
        batch_size=self.batch_size)
    dataset_es_val = DataLoader(
        self.dataset_es["val"], 
        batch_size=self.batch_size)
    loaders = [dataset_ru_val, dataset_es_val]
    return val_loaders

    def test_dataloader(self):
    dataset_ru_test = DataLoader(
        self.dataset_ru["test"], 
        batch_size=self.batch_size)
    dataset_es_test = DataLoader(
        self.dataset_es["test"], 
        batch_size=self.batch_size)
    loaders = [dataset_ru_test, dataset_es_test]
    return test_loaders

In [ ]:
class Model(pl.LightningModule):
  def __init__(self, learning_rate):
    super.__init__()
    self.learning_rate = learning_rate
    self.l1 = nn.Linear(x, y)

  def forward(self, x):
    return torch.relu()

  def training_step(self, batch, batch_idx):
    x, y = batch
    y_hat = self(x)
    loss = F.pairwise_distance(y_hat, y)
    self.log("train_loss", loss, on_epoch=True)
    return loss

  def validation_step(self, batch, batch_idx):
    x, y = batch
    y_hat = self.model(x)
    loss = F.pairwise_distance(y_hat, y)
    self.log("val_loss", loss, on_epoch=True)

  def test_step(self, batch, batch_idx):
    x, y = batch
    y_hat = self.model(x)
    loss = F.pairwise_distance(y_hat, y)
    self.log("test_loss", loss, on_epoch=True)


  def configure_optimizers(self):
    return torch.optim.Adam(self.parameters(), lr=self.lr)

In [ ]:
train_loader = DataLoader(dataset["train"])
trainer = pl.Trainer()

# trainer = pl.Trainer(auto_lr_find=True)
# trainer.tune(model)

# lr_finder = trainer.tuner.lr_find(model)
# lr.finder.results
# fig = lr_finder.plot(suggest=True)
# fig.show()
# new_lr = lr_finder.suggestion()
# model.hparams.lr = new_lr

# trainer = pl.Trainer(auto_scale_batch_size="power")
# trainer.tune(model)

model = Model()

trainer.fit(model, train_loader)
trainer.test(model, test_dataloaders=val_dataloader)
trainer.test(test_dataloaders=test_dataloader)

In [ ]:
# call after training
trainer = pl.Trainer()
trainer.fit(model)
trainer.test(dataloaders=test_dataloader)



In [ ]:
# or call with pretrained model
model = MyLightningModule.load_from_checkpoint(PATH)
trainer = pl.Trainer()
trainer.test(model, dataloaders=test_dataloader)

In [ ]:
# get predictions
PATH = "../saved_model"
my_model = Model.load_from_checkpoint(PATH)
my_model.freeze()
prediction = my_model(new_text)